## การทำตรวจจับ Coin_Segmentation 

1.อ่านภาพจากวิดีโอ และ กล้อง (แต่ละ frame) 

In [2]:
import cv2

cap = cv2.VideoCapture(0) 
# Read from video = type "name of video"
# Read from camera = type 0

while(cap.isOpened()):          # เมื่อมีการเปิดเล่น video : read จะใช้กับ video , isOpened จะใช้กับกล้อง
    ref,frame = cap.read()      # รับข้อมูลเข้ามาเก็บไว้
    cv2.imshow("Show",frame)    # แสดง frame ออมา
    
    if cv2.waitKey(1) & 0xFF == ord('q'):       # ทำการรอคำสั่ง key ที่กดเข้ามาหรือ q แล้วทำการ Break ทิ้งไป
        break
    
cap.release()
cv2.destroyAllWindows()

1.1.แปลงภาพเป็น gray scale และแปลงเป็น binary เข้าถึงการทำงานและวิเคราะห์รูปร่าง

In [ ]:
import cv2

cap = cv2.VideoCapture(0) 

while(cap.isOpened()):          
    ref,frame = cap.read()   
    
    #กำหนดค่า ROI(Region of Interest บริเวรพื้นที่ที่เราสนใจแล้วไปตัดมาในภาพ)
    roi = frame[:1080,0:1920]           # [ความสูง , ความกว้าง]
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)      # แปลงภาพให้เป็นสีเทา
    
    cv2.imshow("Show",gray)    
    if cv2.waitKey(1) & 0xFF == ord('q'):       
        break
    
cap.release()
cv2.destroyAllWindows()

2.Gaussian Blur and Threshold (Gaussian Blur : เพื่อกำจัด noise ทิ้งไปแต่ถ้าทำมากๆก็จะทำให้ภาพ Blur , Threshold : เพื่อแยกภาพพื้นหลังออกจากวัตถุ)

In [ ]:
import cv2

cap = cv2.VideoCapture(0) 

while(cap.isOpened()):          
    ref,frame = cap.read()   
    roi = frame[:1080,0:1920]         
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)     
    
    # ทำการลบ noise ด้วยตัว gaussian blur ด้วยค่า kernel size(ขนาดที่จะแปะลงไปในบริเวณภาพ) 
    # โดยทำการเอา pixels กลบลงไปทำให้เกิดการคลาดเคลื่อนภาพ blur ขึ้น
    gray_blur = cv2.GaussianBlur(gray,(15,15),0)        # GaussianBlur(รูปภาพหลังจาก convert, kernel size, repeat ซ้ำ)
    
    cv2.imshow("Show",gray_blur)                        # ภาพที่แสดงออกมา  จุดด่างๆ หรือ จุด noise ในภาพก็จะโดนลบทิ้ง
    if cv2.waitKey(1) & 0xFF == ord('q'):       
        break
    
cap.release()
cv2.destroyAllWindows()

2.1.แปลงภาพเป็นแบบ Binary และกำหนด Threshold เพื่อแยกออกจากพื้นหลังใช้ Adaptive Thresholding (เลือกค่า Threshold ที่เหมาะสมโดยเลือกมาจากค่า pixels ใกล้เคียงนั้นโดยสามารถใช้ได้ทุกในสถานะการของภาพ เช่น ภาพความสว่างไม่สม่ำเสมอ เพราะ เกิดจากการนำภาพ pixels ใกล้เคียงมาประมวลผล)

In [1]:
import cv2

cap = cv2.VideoCapture(0) 

while(cap.isOpened()):          
    ref,frame = cap.read()   
    roi = frame[:1080,0:1920]         
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)     
    gray_blur = cv2.GaussianBlur(gray,(15,15),0)        
    
    # ในการใช้ตัว Adaptive Threshold เราต้องเลือกใช้ตัว AdaptiveMethod ให้ถูต้อง และเราเลือกทำ Gaussian แทนที่จะทำค่า Mean เพราะฉะนั้นจำเป็นต้องใช้ Adaptive Thresh Gaussian 
    thresh = cv2.adaptiveThreshold(gray_blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,1)           #  adaptiveThreshold(ภาพ frame,ต้องการให้แสดงค่า Threshold ที่ 255(เป็นสีขาว) บริเวณที่เป็นตรงกลางเหรียญ, ตัว adaptive method, ต้องทำการสลับขั้วกันเพราะรอบแรกที่ method ค้นเจอจะเอารอบนอกพื้นที่เหรียญเป็นสีขาวและตัวเหรียญสีดำ เพราะฉะนั้นต้อง invert กลับเพื่อให้ track เหรียญที่เป็นสีขาวขอบนอกสีดำ, blockSize, Constant)
    
    cv2.imshow("Show",thresh)                      
    if cv2.waitKey(1) & 0xFF == ord('q'):       
        break
    
cap.release()
cv2.destroyAllWindows()

3.Morphological Closing (วิเคราะห์รูปร่างโครงสร้างของภาพโดยใช้ Morphology) ทำการ closing เพื่อเติมเต็มช่องว่างโดยทดลองใช้ Array Matrix numpy : 1x1, 2x2, 3x3 ครอบลงไป (ถ้าต้องการวัตถุที่มีโครงสร้างชัดเจนให้กำหนด Array Matrix เยอะขึ้น)

In [4]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0) 

while(cap.isOpened()):          
    ref,frame = cap.read()   
    roi = frame[:1080,0:1920]         
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)     
    gray_blur = cv2.GaussianBlur(gray,(15,15),0)        
    thresh = cv2.adaptiveThreshold(gray_blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,1)          
    
    # kernel = np.ones((1,1),np.uint8)           # สร้าง kernel array matrix เพื่อแปะจุดที่เป็นด่างๆ ให้เติมเต็มให้หมดในเหรียญหรือวัตถุที่หาเจอ / np.ones(ขนาด,type) : สร้างพื้นที่จองไว้
    kernel = np.ones((2,2),np.uint8)
    # kernel = np.ones((3,3),np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=5)            # morphologyEx(ภาพ threshold, operator, สิ่งที่จะนำไปใช้งานตัว array matrix kernel, iterations : repeat การทำซ้ำ)
    
    cv2.imshow("Show",closing)                      
    if cv2.waitKey(1) & 0xFF == ord('q'):       
        break
    
cap.release()
cv2.destroyAllWindows()

4.แสดงรูปร่างและกรองพื้นที่เหรียญ (เจอรูปร่างวงกลมแล้วค่อยวาดทับลงไปบนรูปภาพโดยขนาดก็อิงจากรูปภาพที่พบ)

In [6]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0) 

while(cap.isOpened()):          
    ref,frame = cap.read()   
    roi = frame[:1080,0:1920]         
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)     
    gray_blur = cv2.GaussianBlur(gray,(15,15),0)        
    thresh = cv2.adaptiveThreshold(gray_blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,1)          
    kernel = np.ones((2,2),np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=5)            
    
    result_img = closing.copy()
    contours, hierachy = cv2.findContours(result_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)         # ค้นหารูปร่างของเหรียญ findContours(ภาพที่จะใช้ค้น ,mode(ส่งค่าขอบกลับมาเพื่อหาขอบ) ,mode(ปรับให้ขอบกระชับขึ้นได้สัดส่วน)) โดยจะได้ออกมา 2 ค่าคือ contours : พื้นที่ที่เจอ, hierachy : ชั้นการทำงานของวัตถุ
    counter = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)                 # ทำการเข้าถึงพื้นที่ภายใน contours
        if area < 5000 or area > 35000:             # กำหนดพื้นที่ถายใน contours area ที่ต้องการหา ถ้าพื้นที่น้อย : จะมีการหาขอบที่มากขึ้น, ถ้าพื้นที่พอดี : จะแม่นยำมากยิ่งขึ้น (area ต่ำสุด 2000, area สูงสุด 35000)
            continue
        ellipsis = cv2.fitEllipse(cnt)              # สร้างวงรีขึ้นมาภายใน contours
        cv2.ellipse(roi,ellipsis,(0,255,0),2)       # สร้างวงรีบริเวณที่สนใจและวาดวงรีตามที่ scan เจอ / ellipse(รูปภาพ,วงกลมที่ scan เจอ, สีของวงกลม, ความหนา)
        counter += 1                                # ทำการสร้างตัวนับ
    
    cv2.putText(roi,str(counter),(10,100),cv2.FONT_HERSHEY_SIMPLEX,4,(255,0,0),2,cv2.LINE_AA)
    
    cv2.imshow("Show",roi)                      
    if cv2.waitKey(1) & 0xFF == ord('q'):       
        break
    
cap.release()
cv2.destroyAllWindows()